In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.2/168.2 kB 10.7 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
import joblib
import gradio as gr
import re
import requests
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import nltk
import matplotlib.pyplot as plt

# Download stopwords
nltk.download('stopwords')

# Load pre-trained model and vectorizer
model = joblib.load('fake_news_model.pkl')  # Load your trained model
vectorizer = joblib.load('tfidf_vectorizer.pkl')  # Load your TfidfVectorizer

# Stemming function
ps = PorterStemmer()
def stemming(content):
    content = re.sub('[^a-zA-Z]', ' ', content)
    content = content.lower().split()
    content = [ps.stem(word) for word in content if word not in stopwords.words('english')]
    return ' '.join(content)  # Rejoin words into a single string

# Function to scrape content from a news URL
def fetch_news_content(url):
    try:
        # Send an HTTP GET request to the URL
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # Raise an exception for HTTP errors

        # Parse HTML content with BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract paragraphs (most news articles use <p> tags for content)
        paragraphs = soup.find_all('p')
        news_content = ' '.join([para.get_text() for para in paragraphs])

        return news_content.strip()  # Return the concatenated text
    except Exception as e:
        return f"Error fetching content: {e}"

# Prediction function that integrates scraping and creates a pie chart
def classify_news_from_url(news_url):
    # Fetching news content from the URL
    news_content = fetch_news_content(news_url)

    # Validating fetched content
    if "Error" in news_content or not news_content:
        return "Failed to fetch or process the content. Check the URL.", None

    # Preprocessing the content
    processed_text = stemming(news_content)

    # Predicting using the pre-trained model
    input_data = vectorizer.transform([processed_text])  # Vectorize the preprocessed text
    prediction = model.predict(input_data)[0]  # Predict label
    probabilities = model.predict_proba(input_data)[0]  # Predict probabilities

    # Generate classification result
    result = "Fake News" if prediction == 1 else "Real News"

    # Create a pie chart
    labels = ['Real News', 'Fake News']
    sizes = [probabilities[0], probabilities[1]]
    colors = ['green', 'red']
    explode = (0.1, 0)  # Highlight the first slice (Real News)
    plt.figure(figsize=(5, 5))
    plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%', startangle=140)
    plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

    # Return result and the pie chart
    return result, plt

# Gradio Interface
with gr.Blocks() as iface:
    gr.Markdown(
        """
        # Fake News Detector with Real-Time Web Scraping
        Enter a URL to fetch and classify news as **real** or **fake**. This tool scrapes the article content and uses a pre-trained machine learning model to classify the news.
        """
    )

    with gr.Row():
        with gr.Column(scale=1):

            url_input = gr.Textbox(
                lines=1,
                placeholder="Enter the news article URL here...",
                label="News URL",
                elem_id="news-url"
            )

            submit_button = gr.Button("Submit", elem_id="submit-btn")
            clear_button = gr.Button("Clear", elem_id="clear-btn")


            result_output = gr.Textbox(
                label="Decision",
                placeholder="Result will be displayed here...",
                elem_id="result"
            )

        with gr.Column(scale=2):

            pie_chart = gr.Plot(label="Prediction Breakdown (Pie Chart)")


    submit_button.click(fn=classify_news_from_url, inputs=url_input, outputs=[result_output, pie_chart])


    clear_button.click(fn=lambda: ("", None), inputs=[], outputs=[result_output, pie_chart])

    iface.css = """
        #submit-btn, #clear-btn {
            background-color: green;
            color: white;
        }
        #submit-btn:hover, #clear-btn:hover {
            background-color: darkgreen;
        }
    """

iface.launch()








[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8d1b8e9d56934846ad.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:

from google.colab import files

# Upload files
uploaded = files.upload()

Saving fake_news_model.pkl to fake_news_model.pkl
Saving tfidf_vectorizer.pkl to tfidf_vectorizer.pkl
